In [1]:
import pandas as pd
import datetime
s=pd.Timestamp.today().normalize()  

In [2]:
today=s.strftime("%Y-%m-%d")
print(today)
days_28_period= s - datetime.timedelta(29)
days_28_period=days_28_period.strftime("%Y-%m-%d")
days_28_period

2023-03-24


'2023-02-23'

In [3]:
from pandas import json_normalize
import json
import matplotlib.pyplot as plt
#import requests module
import requests
from requests.auth import HTTPBasicAuth
page=1
sales_data=[]
while(1):
    response = requests.get('https://api.cin7.com/api/v1/SalesOrders?rows=250&where= createdDate<'+str(today)+'T00:00:00Z and createdDate>'+days_28_period+'T23:59:59Z&fields=createdDate,lineItems&page='+str(page),auth = HTTPBasicAuth('DiggsIncUS', '9901f725991a42c4b8ea12917e3c21d9'))
    if not response.json():
        break
    #print (response.json())
    page_data=response.json()
    #separarting each object from the the output array and then appending it sales_Data to have an array of salesrecord objects
    sales_data += page_data
    #end of a single page
    #print("-------------------------------------------------") 
    #increasing the page value to iterate through all pages and get the data
    page+=1 
#print(sales_data)                    


In [4]:
sales_data

[{'createdDate': '2023-03-23T23:33:10Z',
  'lineItems': [{'id': 194954,
    'createdDate': '2023-03-23T23:33:10Z',
    'transactionId': 86734,
    'parentId': 0,
    'productId': 601,
    'productOptionId': 562,
    'integrationRef': '0',
    'sort': 10,
    'code': 'CRA019AAIW',
    'name': 'Revol - I / Ash',
    'option1': 'Ash',
    'option2': 'I',
    'option3': '',
    'qty': 1.0,
    'styleCode': 'CRA019AAIW',
    'barcode': '',
    'sizeCodes': None,
    'lineComments': None,
    'unitCost': 281.2,
    'unitPrice': 595.0,
    'discount': 89.25,
    'qtyShipped': 1.0,
    'holdingQty': 0.0,
    'accountCode': '',
    'stockControl': 'FIFO',
    'stockMovements': [{'batch': None, 'quantity': 1.0, 'serial': None}]},
   {'id': 194956,
    'createdDate': '2023-03-23T23:33:14Z',
    'transactionId': 86734,
    'parentId': 194954,
    'productId': 691,
    'productOptionId': 652,
    'integrationRef': '0',
    'sort': 10,
    'code': 'CRA019AAIW-1',
    'name': 'Intermediate Revol Crat

In [9]:
df_sales = json_normalize(sales_data)
df=df_sales.explode('lineItems').reset_index(drop=True)
df=df.dropna().reset_index(drop=True)
df['code']=''
df['qty']=''
for i in range(len(df)):
    df.loc[i, "code"]=df.loc[i, "lineItems"]['code']
    df.loc[i, "qty"]=int(df.loc[i, "lineItems"]['qty'])
df=df.filter(['createdDate','code','qty'], axis=1)
df

,createdDate,code,qty
0,2023-03-23T23:33:10Z,CRA019AAIW,1
1,2023-03-23T23:33:10Z,CRA019AAIW-1,1
2,2023-03-23T23:33:10Z,CRA019AAIW-2,1
3,2023-03-23T23:33:00Z,PAD025AAIG,1
4,2023-03-23T23:29:06Z,GRO009AASBR,1
...,...,...,...
8281,2023-02-24T00:12:07Z,SC0S0001V04G,1
8282,2023-02-24T00:12:07Z,WSP002AA08I,1
8283,2023-02-24T00:06:11Z,CRA016AAMW,1
8284,2023-02-24T00:06:11Z,PAD020AAMG,1


In [10]:
#accessing the skumapping excel
compare_df=pd.read_excel('/Users/nidhimenon/Downloads/DiggsSKUMapping.xlsx')
compare_df=compare_df.filter(['Diggs SKU', 'Variant'])
compare_df=compare_df.rename(columns={'Diggs SKU':'code'})
#compare_df
#matching the data from sales to the skumapping excel sheet to get product variants and their quantity sold
final_df=pd.merge(df, compare_df, on='code', how='left')
print(final_df)
#final_df.to_excel("/Users/nidhimenon/Desktop/output_nan.xlsx")
final_df=final_df.groupby("Variant")["qty"].sum()
final_df=final_df.sort_values(ascending=False)
#print(final_df)

               createdDate          code qty  \
0     2023-03-23T23:33:10Z    CRA019AAIW   1   
1     2023-03-23T23:33:10Z  CRA019AAIW-1   1   
2     2023-03-23T23:33:10Z  CRA019AAIW-2   1   
3     2023-03-23T23:33:00Z    PAD025AAIG   1   
4     2023-03-23T23:29:06Z   GRO009AASBR   1   
...                    ...           ...  ..   
8281  2023-02-24T00:12:07Z  SC0S0001V04G   1   
8282  2023-02-24T00:12:07Z   WSP002AA08I   1   
8283  2023-02-24T00:06:11Z    CRA016AAMW   1   
8284  2023-02-24T00:06:11Z    PAD020AAMG   1   
8285  2023-02-24T00:06:11Z    PIL004AAMG   1   

                                Variant  
0                     Revol - Int - Ash  
1               Revol - Int - Ash Box 1  
2               Revol - Int - Ash Box 2  
3     Snooz - Intermediate - Light Grey  
4                         Groov - Blush  
...                                 ...  
8281                                NaN  
8282                          Poop Bags  
8283               Revol - Medium - Ash  
828

In [11]:
final_df

Variant
Groov - Turquoise                      841
Spreadable Treat - Gold N Delicious    816
Spreadable Treat - Thats My Jam        564
Groov - Blush                          524
Crate Bowl Attachment Charcoal         339
                                      ... 
Collar - Large - Blush                   1
Collar - Medium - Blush                  1
Collar - Medium - Lilac                  1
Revol - Small - Pine                     1
Collar - Large - Ash                     1
Name: qty, Length: 132, dtype: int64